In [1]:
import pandas as pd

import sqlite3
conn = sqlite3.connect('./ipl_database.db')

# Create a cursor object
cursor = conn.cursor()

cursor.execute("SELECT name FROM sqlite_master WHERE type='table';")
tables = cursor.fetchall()

# Iterate over the table names
for table in tables:
    table_name = table[0]
    print(f"Table: {table_name}")
    sql_query = 'SELECT * FROM '+ table_name+';'
    df = pd.read_sql_query(sql_query, conn)
    xlsx_file_path = "C:/Users/anubr/Python Projects/IPL/assets_xls/"

    # Convert the DataFrame to a excel file
    df.to_excel(xlsx_file_path+table_name+".xlsx", index=False)

    # Print a message indicating the conversion is complete
    print(f"DataFrame has been successfully converted to CSV file: {xlsx_file_path+table_name}.xls")

    
conn.close()

Table: ipl_ball_by_ball
DataFrame has been successfully converted to CSV file: C:/Users/anubr/Python Projects/IPL/assets_xls/ipl_ball_by_ball.xls
Table: ipl_venue_list
DataFrame has been successfully converted to CSV file: C:/Users/anubr/Python Projects/IPL/assets_xls/ipl_venue_list.xls
Table: ipl_venue_innings_1
DataFrame has been successfully converted to CSV file: C:/Users/anubr/Python Projects/IPL/assets_xls/ipl_venue_innings_1.xls
Table: ipl_venue_innings_2
DataFrame has been successfully converted to CSV file: C:/Users/anubr/Python Projects/IPL/assets_xls/ipl_venue_innings_2.xls
Table: ipl_match_list
DataFrame has been successfully converted to CSV file: C:/Users/anubr/Python Projects/IPL/assets_xls/ipl_match_list.xls
Table: ipl_player_list
DataFrame has been successfully converted to CSV file: C:/Users/anubr/Python Projects/IPL/assets_xls/ipl_player_list.xls


In [11]:
import sqlite3
import data_manipulation
def get_batting_positions_id_innings(match_id, innings):
    conn = sqlite3.connect('./ipl_database.db')
    cursor = conn.cursor()

    # SQL query to fetch all unique batters in the specified match ID and innings
    query = '''
    SELECT DISTINCT batter
    FROM ipl_ball_by_ball
    WHERE ID = ? AND innings = ?
    ORDER BY overs, ballnumber
    '''

    cursor.execute(query, (match_id, innings))
    batting_positions = [row[0] for row in cursor.fetchall()]

    conn.close()
    return batting_positions

def get_all_players(match_id):
    conn = sqlite3.connect('./ipl_database.db')
    cursor = conn.cursor()

    # Retrieve the team players from the ipl_match_list table for the given match ID
    cursor.execute("SELECT Team1Players, Team2Players FROM ipl_match_list WHERE ID = ?;", (match_id,))
    team_players = cursor.fetchone()

    conn.close()

    if team_players:
        # Split the string to get the list of players for both teams
        team1_players = team_players[0][1:-1].split(", ")
        team2_players = team_players[1][1:-1].split(", ")
        all_players = team1_players + team2_players
        return all_players
    else:
        print("No match found for the given ID.")
        return None

def get_remaining_players(match_id, innings):
    conn = sqlite3.connect('./ipl_database.db')
    cursor = conn.cursor()

    # Retrieve toss information to determine which team is batting first
    cursor.execute("SELECT TossWinner, TossDecision FROM ipl_match_list WHERE ID = ?;", (match_id,))
    toss_data = cursor.fetchone()
    team1, team2 = None, None
    if toss_data:
        toss_winner, toss_decision = toss_data

        # Determine the team batting first based on toss decision
        if toss_decision.lower() == 'bat':
            batting_team = toss_winner
        else:
            # If the decision is to field, the other team is batting first
            cursor.execute("SELECT Team1Players, Team2Players FROM ipl_match_list WHERE ID = ?;", (match_id,))
            teams_data = cursor.fetchone()
            team1, team2 = teams_data
            batting_team = team1 if toss_winner == team2 else team2

        # Retrieve the players' names from the ipl_match_list table for the batting team
        cursor.execute("SELECT Team1Players, Team2Players FROM ipl_match_list WHERE ID = ?;", (match_id,))
        players_data = cursor.fetchone()

        conn.close()

        if players_data:
            team1_players_str = players_data[0] if batting_team == team1 else players_data[1]
            team2_players_str = players_data[1] if batting_team == team2 else players_data[0]

            # Convert the string format to a list of player names
            batting_team_players = eval(team1_players_str) if batting_team == team1 else eval(team2_players_str)

            # Get the batters who batted
            batters_list = data_manipulation.get_batting_positions_id_innings(match_id, innings)

            # Remove the batters who batted from the list of all players in the batting team
            remaining_players = [player for player in batting_team_players if player not in batters_list]

            return remaining_players
        else:
            print("No match found for the given ID.")
            return None
    else:
        print("No toss information found for the given ID.")
        return None

# Example usage:
match_id = 1312200
innings = 2
remaining_players = get_remaining_players(match_id, innings)
print("Remaining Players:", remaining_players)
print(len(remaining_players))

# Example usage:
match_id = 1312200
innings = 1
remaining_players = get_remaining_players(match_id, innings)
print("Remaining Players:", remaining_players)

Remaining Players: ['YBK Jaiswal', 'JC Buttler', 'SV Samson', 'D Padikkal', 'SO Hetmyer', 'R Ashwin', 'R Parag', 'TA Boult', 'OC McCoy', 'M Prasidh Krishna', 'YS Chahal']
11
Remaining Players: ['M Prasidh Krishna', 'YS Chahal']


In [59]:
import sqlite3
import ast
import data_manipulation

def remaining_players(ID, innings, batting_list):
    conn = sqlite3.connect('./ipl_database.db')
    cursor = conn.cursor()

    cursor.execute("SELECT TossWinner, TossDecision, Team1, Team2, Team1Players, Team2Players FROM ipl_match_list WHERE ID = ?;", (ID,))
    toss_info = cursor.fetchone()
    
    if toss_info:
        TossWinner, TossDecision, Team1, Team2, Team1Players_str, Team2Players_str = toss_info
        
        Team1Players = ast.literal_eval(Team1Players_str)
        Team2Players = ast.literal_eval(Team2Players_str)

        batting_team_players = Team1Players if TossWinner == Team1 else Team2Players

        if innings == 1:
            if TossDecision == "bat":
                remaining_players_list = [player for player in batting_team_players if player not in batting_list]
            else:
                remaining_players_list = [player for player in batting_team_players]
        else:  # innings == 2
            if TossDecision == "bat":
                remaining_players_list = [player for player in batting_team_players]
            else:
                remaining_players_list = [player for player in batting_team_players if player not in batting_list]

        conn.close()
        return remaining_players_list
    else:
        conn.close()
        return []

ID = 1312199
innings = 1
batters_list = data_manipulation.get_batting_positions_id_innings(ID, innings)
remaining_players(ID, innings, batters_list)


['YBK Jaiswal',
 'JC Buttler',
 'SV Samson',
 'D Padikkal',
 'SO Hetmyer',
 'R Parag',
 'R Ashwin',
 'TA Boult',
 'YS Chahal',
 'M Prasidh Krishna',
 'OC McCoy']

In [51]:
import sqlite3
import pandas as pd

def innings_by_innings_list_match_list_details(id_list):
    Date = []
    Venue = []
    Season = []
    MatchNumber = []
    City = []
    conn = sqlite3.connect('./ipl_database.db')
    cursor = conn.cursor()
    for ID in id_list:
        query = '''
        SELECT 
            Date, Venue, Season, MatchNumber, City   
        FROM 
            ipl_match_list
        WHERE 
            ID = ?
        GROUP BY 
            ID;
        '''
        cursor.execute(query, (ID,))
        x = cursor.fetchall()[0]
        Date.append(x[0])
        Venue.append(x[1])
        Season.append(x[2])
        MatchNumber.append(x[3])
        City.append(x[4])
        
   

    conn.close()

    return Date, Venue, Season, MatchNumber, City

def innings_by_innings_list_batting(name, id_list):
    batter,	Runs_Scored, Balls_Faced, Four_Count, Six_Count, Out, Dismissal_Type, fielders_involved, Bowler = [], [], [], [], [], [], [], [], []

    batter_dict = {}
    conn = sqlite3.connect('./ipl_database.db')
    cursor = conn.cursor()
    for ID in id_list:
        query = '''
        SELECT 
            batter AS Name,
            SUM(batsman_run) AS Runs_Scored,
            COUNT(CASE WHEN extra_type IS NULL OR extra_type = 'byes' OR extra_type = 'legbyes' OR extra_type = 'noballs' THEN 1 ELSE NULL END) AS Balls_Faced,
            SUM(CASE WHEN batsman_run = 4 THEN 1 ELSE 0 END) AS Four_Count,
            SUM(CASE WHEN batsman_run = 6 THEN 1 ELSE 0 END) AS Six_Count,
            COUNT(CASE WHEN isWicketDelivery = 1 THEN 1 ELSE NULL END) AS Out,
            MAX(CASE WHEN isWicketDelivery = 1 THEN kind ELSE NULL END) AS Dismissal_Type,
            MAX(CASE WHEN isWicketDelivery = 1 THEN fielders_involved ELSE NULL END) AS fielders_involved,
            MAX(CASE WHEN isWicketDelivery = 1 AND kind <> 'run out' THEN bowler ELSE NULL END) AS Bowler
            
        FROM 
            ipl_ball_by_ball
        WHERE 
            batter = ? AND ID = ?
        GROUP BY 
            batter;
        '''
        cursor.execute(query, (name, ID))
        x = cursor.fetchall()[0]
        if x:
            batter.append(x[0])
            Runs_Scored.append(x[1])
            Balls_Faced.append(x[2])
            Four_Count.append(x[3])
            Six_Count.append(x[4])
            Out.append(x[5])
            Dismissal_Type.append(x[6])
            fielders_involved.append(x[7])
            Bowler.append(x[8])
        else:
            batter.append(None)
            Runs_Scored.append(None)
            Balls_Faced.append(None)
            Four_Count.append(None)
            Six_Count.append(None)
            Out.append(None)
            Dismissal_Type.append(None)
            fielders_involved.append(None)
            Bowler.append(None)
    
    Date, Venue, Season, MatchNumber, City = innings_by_innings_list_match_list_details(id_list)
    batter_dict["Date"] = Date
    batter_dict["MatchNumber"] = MatchNumber
    batter_dict["Season"] = Season
    batter_dict["batter"] = batter
    batter_dict["Runs_Scored"] = Runs_Scored
    batter_dict["Balls_Faced"] = Balls_Faced
    batter_dict["Four_Count"] = Four_Count
    batter_dict["Six_Count"] = Six_Count
    batter_dict["Out"] = Out
    batter_dict["Dismissal_Type"] = Dismissal_Type
    batter_dict["fielders_involved"] = fielders_involved
    batter_dict["Bowler"] = Bowler
    batter_dict["Venue"] = Venue
    batter_dict["City"] = City
    batter_dict["ID"] = id_list
    
    conn.close()
    df = pd.DataFrame(batter_dict)
    df['Date'] = pd.to_datetime(df['Date'])
    df = df.sort_values(by='Date', ascending=True)
    return df

innings_by_innings_list_batting("Shubman Gill", [1254117, 1304103])

,Date,MatchNumber,Season,batter,Runs_Scored,Balls_Faced,Four_Count,Six_Count,Out,Dismissal_Type,fielders_involved,Bowler,Venue,City,ID
0,2021-10-15,Final,2021,Shubman Gill,51,43,6,0,1,lbw,None,DL Chahar,Dubai International Cricket Stadium,Dubai,1254117
1,2022-05-10,57,2022,Shubman Gill,63,49,7,0,0,None,None,None,Maharashtra Cricket Association Stadium,Pune,1304103
